<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Bayesian Linear Mixed Models (Genomic Data)</font> 
</div>

In [1]:
;pwd

/Users/haocheng/Github/JWAS.jl/docs/notebooks_v0.3


In [2]:
include("../../src/JWAS.jl")

JWAS

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [3]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [4]:
phenofile  = Datasets.dataset("example","phenotypes.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:00

Finished!



calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


In [5]:
head(phenotypes)

,ID,y1,y2,y3,x1,x2,x3,dam
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a2,-0.6,4.9,0.88,0.3,1,f,0
3,a3,-2.07,3.19,0.73,0.7,2,f,0
4,a4,-2.63,6.97,-0.83,0.6,1,m,a2
5,a5,2.31,3.5,-1.52,0.4,2,m,a2
6,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univariate Linear Mixed Model (Genomic data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [31]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [32]:
R      = 1.0
model1 = build_model(model_equation1,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [33]:
set_covariate(model1,"x2");

<button type="button" class="btn btn-lg btn-primary">Step 5: Build Random or Fixed Effects</button> 

In [34]:
G1 = 1.0
G2 = eye(2)
set_random(model1,"x2",G1);
set_random(model1,"x3",G1);
set_random(model1,"ID dam",pedigree,G2);

<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [35]:
G3 =1.0
add_genotypes(model1,genofile,G3,separator=',',G_is_marker_variance=true);

5 markers on 10 individuals were added.


In [36]:
outputMCMCsamples(model1,"x3")

<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [37]:
out1=runMCMC(model1,phenotypes,chain_length=5000,methods="BayesB",output_samples_frequency=100);




MCMC Information:

methods                                      BayesB
chain_length                                   5000
burnin                                            0
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                                               0.0
estimatePi                                    false

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





running MCMC for BayesB ...100%|████████████████████████| Time: 0:00:01


In [13]:
keys(out1)

Base.KeyIterator for a Dict{Any,Any} with 3 entries. Keys:
  "Posterior mean of marker effects"
  "Posterior mean of location parameters"
  "MCMC samples for: 1:x3"

In [14]:
out1["MCMC samples for: 1:x3"]

51×2 Array{Any,2}:
   "1:x3 : m"    "1:x3 : f"
 -1.29018      -0.791645   
 -0.576721      0.462971   
  0.847065     -0.0730476  
  0.943571      0.22769    
 -1.40503      -0.24283    
  1.9577        1.57373    
  0.691723     -0.463638   
  0.32902      -0.27348    
  0.012153     -0.910058   
 -0.0437385     1.115      
 -0.0276657     1.15303    
 -0.806354     -0.782516   
  ⋮                        
  1.52325      -2.61419    
 -0.54575      -1.04289    
 -0.571396     -0.0172729  
  1.30215      -0.989892   
  0.0505139     0.269762   
 -0.521277     -0.62662    
  0.396045     -0.445992   
 -1.26731       1.12148    
  0.916822     -0.0603681  
 -0.569464     -0.39921    
 -0.858377      0.825486   
  1.2156        0.297428   

In [15]:
out1["Posterior mean of location parameters"]

33×2 Array{Any,2}:
 "1:intercept : intercept"  -18.3513    
 "1:x1*x3 : 0.9 * m"         17.8156    
 "1:x1*x3 : 0.3 * f"         17.5648    
 "1:x1*x3 : 0.7 * f"         16.2725    
 "1:x1*x3 : 0.6 * m"         16.0125    
 "1:x1*x3 : 0.4 * m"         17.7696    
 "1:x1*x3 : 5.0 * f"         20.5157    
 "1:x1*x3 : 0.5 * f"         16.7506    
 "1:x1*x3 : 0.3 * m"         15.142     
 "1:x1*x3 : 0.2 * m"         18.5722    
 "1:x2 : x2"                  0.0914076 
 "1:x3 : m"                  -0.00792315
 "1:x3 : f"                   0.00473313
 ⋮                                      
 "1:ID : a10"                 0.256301  
 "1:ID : a9"                 -0.297277  
 "1:dam : a1"                -0.292514  
 "1:dam : a3"                -0.382114  
 "1:dam : a6"                -0.671305  
 "1:dam : a7"                -0.361783  
 "1:dam : a2"                 0.654975  
 "1:dam : a4"                 0.191197  
 "1:dam : a8"                -0.23995   
 "1:dam : a5"                 0.172734

In [16]:
out1["MCMC samples for: 1:x3"]

51×2 Array{Any,2}:
   "1:x3 : m"    "1:x3 : f"
 -1.29018      -0.791645   
 -0.576721      0.462971   
  0.847065     -0.0730476  
  0.943571      0.22769    
 -1.40503      -0.24283    
  1.9577        1.57373    
  0.691723     -0.463638   
  0.32902      -0.27348    
  0.012153     -0.910058   
 -0.0437385     1.115      
 -0.0276657     1.15303    
 -0.806354     -0.782516   
  ⋮                        
  1.52325      -2.61419    
 -0.54575      -1.04289    
 -0.571396     -0.0172729  
  1.30215      -0.989892   
  0.0505139     0.269762   
 -0.521277     -0.62662    
  0.396045     -0.445992   
 -1.26731       1.12148    
  0.916822     -0.0603681  
 -0.569464     -0.39921    
 -0.858377      0.825486   
  1.2156        0.297428   

In [17]:
out2=runMCMC(model1,phenotypes,chain_length=5000,output_samples_frequency=100);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   5000
burnin                                            0
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





LoadError: [91mPlease build your model again using the function build_model().[39m